In [ ]:
!pip install bigartm10
!pip install deeppavlov
!python -m deeppavlov install morpho_ru_syntagrus_pymorphy 
!python -m deeppavlov install squad_bert
!pip install mysmallutils
!pip install stop_words

In [ ]:
!pip uninstall pandas

In [ ]:
!pip install pandas

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import re
import artm
import nltk
import json
import requests
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from os.path import join
from nltk.util import ngrams
from itertools import accumulate
from nltk.corpus import stopwords
from stop_words import get_stop_words
from gensim.corpora import Dictionary
from string import digits, punctuation
from nltk.tokenize import sent_tokenize
from nltk import FreqDist, word_tokenize
from plotly.subplots import make_subplots
from deeppavlov import build_model, configs
from mysutils.text import remove_urls as delete_urls
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Dataset creation

In [ ]:
access_token = 'f52e30cbf52e30cbf52e30cbc4f571d0b5ff52ef52e30cbaa908ed0c563102e7ecf2e65'

In [ ]:
domains = ['art_and_stories', 'space_live', 'nplusone', 'mental_engineering', 'formula_1_fan', 'eco.society', 'gorkiy_media']
topics = ['art', 'space', 'science', 'mental_health', 'motorsport', 'ecology', 'literature']

In [ ]:
posts_url = 'https://api.vk.com/method/wall.get?domain={domain}' \
                                              '&count=100' \
                                              '&offset={offset}' \
                                              '&access_token={access_token}' \
                                              '&v=5.130'

In [ ]:
posts = {'domain': [],
         'text': [],
         'marked_as_ads': []}

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/Diploma'

In [ ]:
for domain, topic in zip(domains, topics):
    for offset in range(0, 10000, 100):
        print(offset)
        posts_url_formatted = posts_url.format(domain=domain,
                                            offset=offset,
                                            access_token=access_token)
        wall = requests.get(posts_url_formatted)
        for post in wall.json()['response']['items']:
            if len(post['text']) > 500:
                posts['domain'].append(domain)
                posts['text'].append(post['text'])
                posts['marked_as_ads'].append(post['marked_as_ads'])

    posts = pd.DataFrame(posts)
    posts = posts[posts['marked_as_ads'] == 0]
    posts.drop('marked_as_ads', axis=1, inplace=True)
    posts.reset_index(inplace=True, drop=True)
    posts.to_csv(join(path, topic + '.csv'), index=False)  
    posts = pd.read_csv(join(path, topic + '.csv'))

In [ ]:
art = pd.read_csv(join(path, 'art.csv'))
space = pd.read_csv(join(path, 'space.csv'))
science = pd.read_csv(join(path, 'science.csv'))
mental_health = pd.read_csv(join(path, 'mental_health.csv'))
motorsport = pd.read_csv(join(path, 'motorsport.csv'))
ecology = pd.read_csv(join(path, 'ecology.csv'))
literature = pd.read_csv(join(path, 'literature.csv'))

In [ ]:
space

,domain,text
0,space_live,"Первый человек, вышедший в открытый космос – А..."
1,space_live,Лунный заговор в мировом кинематографе: расска...
2,space_live,В последние годы ведётся много разговоров о по...
3,space_live,Скоро на обратную сторону Луны упадёт часть Fa...
4,space_live,«Луне-9» — 56 лет!\n\n3 февраля 1966 года сове...
...,...,...
4968,space_live,Комета L4 PANSTARRS растет и набирает яркость....
4969,space_live,Хотите предложить название для экзопланеты? Вс...
4970,space_live,У Земли нашли третий радиационный пояс!\n\nРад...
4971,space_live,Вращение чёрных дыр\n\nАмериканский орбитальны...


Text processing

In [ ]:
model = build_model(configs.morpho_tagger.BERT.morpho_ru_syntagrus_bert,
                    download=True)

nltk.download('stopwords', quiet=True)
stop_words = stopwords.words('russian')
stop_words.extend(get_stop_words('ru'))
stop_words.extend(['что', 'на', 'это', 'свой', 'из-за', 'вв', 'гг' 'г', 'совсем', 'сами', 'самих', 'их', 'бoлee',
                   'кто-то', 'что-то', 'этa', 'вроде', 'всякий', 'никакой', 'самый', 'среди', 'наверно', 'тд'])
stop_words = list(set(stop_words))

punctuation += '№«»“”×…—'

In [ ]:
def lemmatization(text):
    lemmas = [lemma.split('\t')[2] 
              for sentence_piece in model([text])
              for lemma in sentence_piece.split('\n')
              if lemma]
    return ' '.join(lemmas)

In [ ]:
def delete_emojis(text):
    characters = re.compile("["
                        u"\U0001F600-\U0001F64F"
                        u"\U0001F300-\U0001F5FF"
                        u"\U0001F680-\U0001F6FF"
                        u"\U0001F1E0-\U0001F1FF"
                        u"\U00002500-\U00002BEF"
                        u"\U00002702-\U000027B0"
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        u"\U0001f926-\U0001f937"
                        u"\U00010000-\U0010ffff"
                        u"\u2640-\u2642" 
                        u"\u2600-\u2B55"
                        u"\u200d"
                        u"\u23cf"
                        u"\u23e9"
                        u"\u231a"
                        u"\ufe0f"
                        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(characters, '', text)

In [ ]:
def delete_hashtags(text):
    hashtags = re.findall(r'\B[#@]\w*[a-zA-Zа-яА-Я]+\w*', text)
    for hashtag in hashtags:
        text = re.sub(hashtag, '', text)
    return text

In [ ]:
def delete_stop_words(texts):
    return [' '.join([word for word in text.split() if word not in stop_words])
                      for text in texts]

In [ ]:
def text_processing(texts,
                    lemmatize=True,
                    remove_punctuation=True,
                    remove_urls=True,
                    remove_digits=True,
                    remove_stop_words=True,
                    remove_single_chars=True,
                    remove_emojis=True,
                    remove_hashtags=True):
    
    texts = [text.strip(' "\n\t\r') for text in texts]
    texts = [text.translate(str.maketrans('\n\t\r', '   ')) for text in texts]
    
    if remove_stop_words:
        texts = delete_stop_words(texts)

    if remove_urls:
        texts =  [delete_urls(text) for text in texts]

    if remove_punctuation:
        texts = [text.replace('-', ' ') for text in texts]

    if remove_emojis:
        texts = [delete_emojis(text) for text in texts]

    if remove_hashtags: # and mentions
        texts = [delete_hashtags(text) for text in texts]

    if lemmatize:
        texts = [lemmatization(text) for text in texts]

    if remove_digits:
        texts = [text.translate(str.maketrans('', '', digits)) for text in texts]
    
    if remove_stop_words:
        texts = delete_stop_words(texts)

    if remove_punctuation:
        texts = [text.translate(str.maketrans('', '', punctuation)) for text in texts]

    if remove_single_chars:
        texts = [' '.join([sentence_piece for sentence_piece in text.split()
                           if len(sentence_piece) > 1]) for text in texts]

    return texts

Creation of batches

In [ ]:
def create_sentence_batches(text):
    sentences = sent_tokenize(text)
    sentence_batches = []
    while sentences:
        max_count = sum(size + sep <= 1000 for sep, size in enumerate(accumulate(map(len, sentences))))
        sentence_batches.append(' '.join(sentences[:max_count]))
        sentences = sentences[max_count:]
    return sentence_batches

In [ ]:
def create_n_grams(text, n_gram_range):
    n_grams = []
    for n in range(n_gram_range[0], n_gram_range[1] + 1):
        n_grams.extend(['_'.join(n_gram) for n_gram in ngrams(text.split(), n)])
    return n_grams

In [ ]:
def create_batches(texts, n_gram_range, path):
    sentence_batches = [create_sentence_batches(text) for text in texts]
    texts = [text_processing(sentence_batch) for sentence_batch in sentence_batches]
    texts = [' '.join(text) for text in texts]
    texts = [create_n_grams(text, n_gram_range) for text in texts]
    token_frequencies = [FreqDist(text) for text in texts]
    token_frequencies = [[(key + ':' + str(value)) for key, value in freq_dist.items()] for freq_dist in token_frequencies]

    with open(join(path, 'data'), 'w') as file:
        for n in range(len(texts)):
            file.write(f'doc{n} {" ".join(token_frequencies[n])}\n')
    
    with open(join(path, 'preprocessed_texts'), 'w') as file:
        for n in range(len(texts)):
            file.write(f'{" ".join(texts[n])}\n')

    batch_vectorizer = artm.BatchVectorizer(data_path=join(path, 'data'),
                                            data_format='vowpal_wabbit',
                                            target_folder=join(path, 'batches'))
    return batch_vectorizer, texts

In [ ]:
def load_batches(path):
    batch_vectorizer = artm.BatchVectorizer(data_path=join(path, 'data'),
                                            data_format='vowpal_wabbit',
                                            target_folder=join(path, 'batches'))
    
    with open(join(path, 'preprocessed_texts'), 'r') as file:
        texts = file.readlines()
    texts = [text.strip('\n').split() for text in texts]
    
    return batch_vectorizer, texts

In [ ]:
# batch_vectorizer, texts = create_batches(arts, (1, 3), path)

In [ ]:
batch_vectorizer, texts = load_batches(path)

Model parameters

In [ ]:
param_dict = {'domain': ['art', 'art'],
              '#_topics': [25, 50],
              'SparsePhiSubject': [],
              'SparsePhiBackground': [],
              'SparseThetaSubject': [],
              'SparseThetaBackground': [],
              'DecorrelatorPhi': []}

Model fitting

In [ ]:
def create_topic_names(n_subject_topics, n_background_topics):
    subject_topics = ['subject topic #' + str(n) for n in range(1, n_subject_topics + 1)]
    background_topics = ['background topic #' + str(n) for n in range(1, n_background_topics + 1)]
    return subject_topics, background_topics

In [ ]:
subject_topics, background_topics = create_topic_names(10, 2)

In [ ]:
def fit_model(batch_vectorizer,
              num_document_passes,
              num_collection_passes,
              scores, regularizers,
              subject_topics,
              background_topics):

    topics = subject_topics.copy()
    topics.extend(background_topics)

    model_artm = artm.ARTM(topic_names=topics,
                           cache_theta=True,
                           scores=scores,
                           regularizers=regularizers,
                           num_document_passes=num_document_passes)

    model_artm.initialize(dictionary=batch_vectorizer.dictionary)

    model_artm.fit_offline(batch_vectorizer=batch_vectorizer,
                           num_collection_passes=num_collection_passes)
    
    return model_artm

In [ ]:
regularizers = [artm.SmoothSparsePhiRegularizer(name='SparsePhiSubject', topic_names=subject_topics, tau=-0.5),
                artm.SmoothSparsePhiRegularizer(name='SparsePhiBackground', topic_names=background_topics, tau=0.5),
                artm.SmoothSparseThetaRegularizer(name='SparseThetaSubject', topic_names=subject_topics, tau=-2.0),
                artm.SmoothSparseThetaRegularizer(name='SparseThetaBackground', topic_names=background_topics, tau=2.0),
                artm.DecorrelatorPhiRegularizer(name='DecorrelatorPhi', topic_names=subject_topics, tau=100000.0)]

In [ ]:
scores = [artm.PerplexityScore(name='PerplexityScore', dictionary=batch_vectorizer.dictionary),
          artm.SparsityPhiScore(name='SparsityPhiScoreSubject', topic_names=subject_topics),
          artm.SparsityPhiScore(name='SparsityPhiScoreBackground', topic_names=background_topics),
          artm.SparsityThetaScore(name='SparsityThetaScoreSubject', topic_names=subject_topics),
          artm.SparsityThetaScore(name='SparsityThetaScoreBackground', topic_names=background_topics),
          artm.TopicKernelScore(name='TopicKernelScore', probability_mass_threshold=0.3),
          artm.BackgroundTokensRatioScore(name='BackgroundTokensRatioScore', delta_threshold=0.3),
          artm.TopTokensScore(name='TopTokensScore', num_tokens=20)]

In [ ]:
model_artm = fit_model(batch_vectorizer, 30, 9, scores, regularizers, subject_topics, background_topics) 

In [ ]:
def to_pad(words, target_len):
    return words[:target_len] + [' '] * (target_len - len(words))

In [ ]:
to_dataframe = model_artm.score_tracker['TopTokensScore'].last_tokens
for key, value in to_dataframe.items():
    to_dataframe[key] = to_pad(value, 20)

In [ ]:
pd.DataFrame(to_dataframe)

,subject topic #1,subject topic #2,subject topic #3,subject topic #4,subject topic #5,subject topic #6,subject topic #7,subject topic #8,subject topic #9,subject topic #10,background topic #1,background topic #2
0,зевс,ньюман,будда,нью,изабелла,зосима,скуол,тая,адонис,дуано,картина,картина
1,учитель,фотограф,шалот,такси,брат,молитва,тинторетто,героиня,гога,король,художник,художник
2,потоп,врубель,статуя,гофман,пандора,пустыня,ди_сан,такао,ван_гога,шевалье,работа,смерть
3,сом,арнольд,истина,невеста,лоренцо,господь,скуол_ди_сан,цунамунэ,автопортрет,снимок,святой,масло
4,ромбоутс,йорк,хогарт,пиросманить,шило,видеть,скуол_ди,единорог,ван_дейк,репин,холст,размер
5,теодор,нью_йорк,зеркало,вакх,зло,грешный,зал,чёрный,дейк,москва,размер,работа
6,библия,арнольд_ньюман,серов,государственный_третьяковский_галерея,лентулов,преподобный,сан,офелия,афродита,пимоненко,масло,масло_размер
7,караваджо,сын,патриция,уэйн,дейнека,церковь,сан_рокко,младенец,астарта,поцелуй,сантиметр,сантиметр
8,инвалид_война,белый,кот,дельво,климт,иордан,ди_сан_рокко,мадонна,богиня,альберт,искусство,холст
9,теодор_ромбоутс,миро,гравюра,нико,море,ради,рокко,далее,бедность,илья,холст_масло,искусство


In [ ]:
topics = [model_artm.score_tracker['TopTokensScore'].last_tokens[topic_name] for topic_name in model_artm.topic_names]
topics = topics[:10]

In [ ]:
def save_topics(model_artm, path):
    with open(join(path, 'topics.json'), 'w', encoding='utf-8') as file:
        json.dump(model_artm.score_tracker['TopTokensScore'].last_tokens,
                  file,
                  ensure_ascii=False,
                  indent=4)

Coherence, PMI, NPMI

In [ ]:
def calculate_coherence(topics, corpus, dictionary, texts, type_):
    coherence_model = CoherenceModel(topics=topics,
                                     corpus=corpus,
                                     dictionary=dictionary,
                                     texts=texts,
                                     coherence=type_)
    return coherence_model.get_coherence()

Topic Significance

In [ ]:
def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

Uniform Distribution Over Words (W-Uniform)

In [ ]:
def ts_uniform(topic_word_dist):
    n_words = topic_word_dist.shape[0]
    w_uniform = np.ones(n_words) / n_words
    uniform_distances_kl = [kl_divergence(p, w_uniform) for p in topic_word_dist.T]
    return uniform_distances_kl

Vacuous Semantic Distribution (W-Vacuous)

In [ ]:
def ts_vacuous(doc_topic_dist, topic_word_dist):
    n_words = topic_word_dist.shape[0]
    n_tokens = np.sum([len(text) for text in texts])
    p_k = np.sum(doc_topic_dist, axis=1) / n_tokens
    w_vacauous = np.sum(topic_word_dist * np.tile(p_k, (n_words, 1)), axis=1)
    vacauous_distances_kl = [kl_divergence(p, w_vacauous) for p in topic_word_dist.T]
    return vacauous_distances_kl

Background Distribution (D-BGround)

In [ ]:
def ts_bground(doc_topic_dist):
    n_documents = doc_topic_dist.shape[1]
    d_bground = np.ones(n_documents) / n_documents
    d_bground_distances_kl = [kl_divergence(p.T, d_bground) for p in doc_topic_dist]
    return d_bground_distances_kl

SwitchP

In [ ]:
def switchp(phi, texts):
    words = phi.index.to_list()
    words = [word[1] for word in words]
    max_topic_word_dist = np.argmax(phi.to_numpy(), axis=1)
    max_topic_word_dist = dict(zip(words, max_topic_word_dist))
    
    switchp_scores = []
    
    for text in texts:
        mapped_text = [max_topic_word_dist[word] for word in text]
        switches = (np.diff(mapped_text) != 0).sum()
        switchp_scores.append(switches / (len(mapped_text) - 1))
    return switchp_scores

In [ ]:
def print_measures(model_artm, texts, topics):
    id2word = Dictionary(texts)
    corpus = [id2word.doc2bow(text) for text in texts]
    print(f'Coherence: {calculate_coherence(topics, corpus, id2word, texts, "u_mass")}')
    print(f'PMI: {calculate_coherence(topics, corpus, id2word, texts, "c_uci")}')
    print(f'NPMI: {calculate_coherence(topics, corpus, id2word, texts, "c_npmi")}')

    # 'Theta' (Θ) - rows representing a distribution of topics over documents.   
    # 'Phi' (Φ) - rows representing a distribution of words over topics.
    topic_word_dist = model_artm.get_phi().to_numpy()[:, :10]
    doc_topic_dist = model_artm.get_theta().to_numpy()[:10, :]

    print(f'Topic Significance - Uniform Distribution Over Words: {np.mean(ts_uniform(topic_word_dist))}')
    print(f'Topic Significance - Vacuous Semantic Distribution: {np.mean(ts_vacuous(doc_topic_dist, topic_word_dist))}')
    print(f'Topic Significance - Background Distribution: {np.mean(ts_bground(doc_topic_dist))}')

    switchp_scores = switchp(model_artm.get_phi(), texts)
    print(f'SwitchP mean: {np.mean(switchp_scores)}')

    print(f'Sparsity Phi Score Subject: {model_artm.score_tracker["SparsityPhiScoreSubject"].last_value:.3f}')
    print(f'Sparsity Theta Score Subject: {model_artm.score_tracker["SparsityThetaScoreSubject"].last_value:.3f}')
    print(f'Kernel contrast: {model_artm.score_tracker["TopicKernelScore"].last_average_contrast:.3f}')
    print(f'Kernel purity: {model_artm.score_tracker["TopicKernelScore"].last_average_purity:.3f}')
    print(f'Perplexity: {model_artm.score_tracker["PerplexityScore"].last_value:.3f}')
    print(f'Background Tokens Ratio Score: {model_artm.score_tracker["BackgroundTokensRatioScore"].last_value:.3f}')

In [ ]:
print_measures(model_artm, texts, topics)

Coherence: -15.300875950796533
PMI: -10.953407603269863
NPMI: -0.34015853413559316
Topic Significance - Uniform Distribution Over Words: 5.927156602050298
Topic Significance - Vacuous Semantic Distribution: 11.242843627929688
Topic Significance - Background Distribution: 1.7692274866064257
SwitchP mean: 0.25744934990231666
Sparsity Phi Score Subject: 0.996
Sparsity Theta Score Subject: 0.893
Kernel contrast: 0.638
Kernel purity: 0.919
Perplexity: 18660.270
Background Tokens Ratio Score: 0.044


In [ ]:
def draw_artm_metrics(model_artm):
    xs = list(range(len(model_artm.score_tracker['PerplexityScore'].value)))
    figure = make_subplots(rows=4, cols=2,
                           subplot_titles=('Perplexity',
                                           'Background Tokens Ratio',
                                           'Kernel Contrast',
                                           'Kernel Purity',
                                           'Sparsity Phi Score Subject',
                                           'Sparsity Phi Score Background',
                                           'Sparsity Theta Score Subject',
                                           'Sparsity Theta Score Background'))

    figure.add_trace(go.Scatter(x=xs, y=model_artm.score_tracker['PerplexityScore'].value),
                     row=1, col=1)
    figure.add_trace(go.Scatter(x=xs, y=model_artm.score_tracker['BackgroundTokensRatioScore'].value),
                     row=1, col=2)
    figure.add_trace(go.Scatter(x=xs, y=model_artm.score_tracker['TopicKernelScore'].average_contrast),
                     row=2, col=1)
    figure.add_trace(go.Scatter(x=xs, y=model_artm.score_tracker['TopicKernelScore'].average_purity),
                     row=2, col=2)
    figure.add_trace(go.Scatter(x=xs, y=model_artm.score_tracker['SparsityPhiScoreSubject'].value),
                     row=3, col=1)
    figure.add_trace(go.Scatter(x=xs, y=model_artm.score_tracker['SparsityPhiScoreBackground'].value),
                     row=3, col=2)
    figure.add_trace(go.Scatter(x=xs, y=model_artm.score_tracker['SparsityThetaScoreSubject'].value),
                     row=4, col=1)
    figure.add_trace(go.Scatter(x=xs, y=model_artm.score_tracker['SparsityThetaScoreBackground'].value),
                     row=4, col=2)

    figure.update_layout(height=900, width=1100, showlegend=False, font_size=10)
    # figure.update_annotations(font_size=11)
    figure.show()

In [ ]:
draw_artm_metrics(model_artm)

Space documents

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/Diploma/space/'
#batch_vectorizer, texts = create_batches(space.sample(100).text.to_list(), (1, 3), path)
batch_vectorizer, texts = load_batches(path)

In [ ]:
subject_topics, background_topics = create_topic_names(10, 2)

In [ ]:
regularizers = [artm.SmoothSparsePhiRegularizer(name='SparsePhiSubject', topic_names=subject_topics, tau=-0.5),
                artm.SmoothSparsePhiRegularizer(name='SparsePhiBackground', topic_names=background_topics, tau=2),
                artm.SmoothSparseThetaRegularizer(name='SparseThetaSubject', topic_names=subject_topics, tau=-0.5),
                artm.SmoothSparseThetaRegularizer(name='SparseThetaBackground', topic_names=background_topics, tau=0.2),
                artm.DecorrelatorPhiRegularizer(name='DecorrelatorPhi', topic_names=subject_topics, tau=1000000.0)]

In [ ]:
scores = [artm.PerplexityScore(name='PerplexityScore', dictionary=batch_vectorizer.dictionary),
          artm.SparsityPhiScore(name='SparsityPhiScoreSubject', topic_names=subject_topics),
          artm.SparsityPhiScore(name='SparsityPhiScoreBackground', topic_names=background_topics),
          artm.SparsityThetaScore(name='SparsityThetaScoreSubject', topic_names=subject_topics),
          artm.SparsityThetaScore(name='SparsityThetaScoreBackground', topic_names=background_topics),
          artm.TopicKernelScore(name='TopicKernelScore', probability_mass_threshold=0.3),
          artm.BackgroundTokensRatioScore(name='BackgroundTokensRatioScore', delta_threshold=0.3),
          artm.TopTokensScore(name='TopTokensScore', num_tokens=20)]

In [ ]:
model_artm = fit_model(batch_vectorizer, 20, 10, scores, regularizers, subject_topics, background_topics) 

In [ ]:
to_dataframe = model_artm.score_tracker['TopTokensScore'].last_tokens
for key, value in to_dataframe.items():
    to_dataframe[key] = to_pad(value, 20)

In [ ]:
pd.DataFrame(to_dataframe)

,subject topic #1,subject topic #2,subject topic #3,subject topic #4,subject topic #5,subject topic #6,subject topic #7,subject topic #8,subject topic #9,subject topic #10,background topic #1,background topic #2
0,allen_probe,нептун,сверхлёгкий_ракета,пулковский,вмз,сириуссат,глубина,отсек,пулковский_обсерватория,хвост,космический,звезда
1,allen,олимпиада,производство,пыль,протон,origin,bz,мкс,картина,договор,ракета,космический
2,van_allen,слияние,зондирование,пулково,лина_индастриал,космический_погода,атмосферный_вихрь,gateway,квартира,метеорит,компания,телескоп
3,стратосфера,technologies,дистанционный_зондирование,волна,лина,углерод,джинса,груз,определение,or,спутник,планета
4,защита,pi,разрабатывать,красный_смещение,сиппель,физика,формирование,пилотировать,антенна,харон,запуск,аппарат
5,посадочный,pi_mensae,канада,страна,нейтральный,спутник_сириуссат,внешний,bigelow,слабый,бенуа_февля,проект,обсерватория
6,операция,родительский_звезда,канадский,построить,гипотеза,соединение,пейзаж,воздух,поверхность_марс,индийский_космический_агентство,двигатель,наблюдение
7,искусственный,hd,услуга,erosita,завод,органический_соединение,формироваться,жилой_отсек,vlt,пиар,ступень,земля
8,пояс_ван,родительский,google,санкт,предприятие,доля,происхождение,захват,ночь,утверждать,новый,научный
9,ван,центральный,maxar_technologies,город,ионизовать,большой_взрыв,спутник_юпитер,space,выполнять,gault,звезда,система


In [ ]:
save_topics(model_artm, path)

In [ ]:
topics = [model_artm.score_tracker['TopTokensScore'].last_tokens[topic_name] for topic_name in model_artm.topic_names]
topics = topics[:10]

In [ ]:
topics

In [ ]:
print_measures(model_artm, texts, topics)

Coherence: -12.838331997607327
PMI: -10.186839878080097
NPMI: -0.3270208952469375
Topic Significance - Uniform Distribution Over Words: 6.5964914883989225
Topic Significance - Vacuous Semantic Distribution: 11.958256721496582
Topic Significance - Background Distribution: 0.5793891739640598
SwitchP mean: 0.20729682415656586
Sparsity Phi Score Subject: 0.998
Sparsity Theta Score Subject: 0.817
Kernel contrast: 0.655
Kernel purity: 0.991
Perplexity: 25376.715
Background Tokens Ratio Score: 0.018


In [ ]:
draw_artm_metrics(model_artm)